## File to implement nn with wandb
(note that we can't run wandb in terminal)

In [3]:
# file to load the nn

import pandas as pd
import numpy as np
import os
import tensorflow as tf
from keras import layers
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import wandb
from wandb.keras import WandbCallback

# read in our data
DATA_DIR = '/home/oscar47/Desktop/astro101/data/g_band/var_output/v0.1.0'

# check if keras recognizes gpu
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

train_x_ds = np.load(os.path.join(DATA_DIR, 'train_x_ds.npy'))
val_x_ds = np.load(os.path.join(DATA_DIR, 'val_x_ds.npy'))
train_y_ds = np.load(os.path.join(DATA_DIR, 'train_y_ds.npy'))
val_y_ds = np.load(os.path.join(DATA_DIR, 'val_y_ds.npy'))

input_shape = train_x_ds[0].shape
output_len = len(train_y_ds[0])

# build model functions--------------------------------
def build_model(size1, size2, size3, size4, size5, dropout, learning_rate):
    model = Sequential()

    model.add(layers.Dense(size1))
    model.add(layers.Dense(size2))
    model.add(layers.Dense(size3))
    model.add(layers.Dense(size4))
    model.add(layers.Dense(size5))

    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(output_len))

    # return len of class size
    model.add(layers.Dense(output_len))
    model.add(layers.Activation('softmax'))

    optimizer = Adam(learning_rate = learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy')

    return model


def train(config=None):
    with wandb.init(config=config):
    # If called by wandb.agent, as below,
    # this config will be set by Sweep Controller
      config = wandb.config

      #pprint.pprint(config)

      #initialize the neural net; 
      global model
      model = build_model(config.size_1,  config.size_2, config.size_3, 
              config.size_4, config.size_5, 
              config.dropout, config.learning_rate)
      
      #now run training
      history = model.fit(
        train_x_ds, train_y_ds,
        batch_size = config.batch_size,
        validation_data=(val_x_ds, val_y_ds),
        epochs=config.epochs,
        callbacks=[WandbCallback()] #use callbacks to have w&b log stats; will automatically save best model                     
      )

def train_manual():
    global model
    model = build_model(128, 128, 128, 
            128, 128, 
            .1, .001)
    
    #now run training
    history = model.fit(
    train_x_ds, train_y_ds,
    batch_size = 64,
    validation_data=(val_x_ds, val_y_ds),
    epochs=10
    )

# set dictionary with random search; optimizing val_loss--------------------------
sweep_config= {
    'method': 'random',
    'name': 'val_accuracy',
    'goal': 'maximize'
}

sweep_config['metric']= 'val_accuracy'

# now name hyperparameters with nested dictionary
# parameters_dict = {
#     'epochs': {
#        'distribution': 'int_uniform',
#        'min': 10,
#        'max': 20
#     },
#     # for build_dataset
#      'batch_size': {
#        'distribution': 'q_log_uniform',  #we want to specify a distribution type to more efficiently iterate through these hyperparams
#        'q': 8,
#        'min': np.log(64),
#        'max': np.log(256)
#     },
#     'size_1': {
#        'distribution': 'q_log_uniform',
#        'q': 8,
#        'min': np.log(64),
#        'max': np.log(256)
#     },
#     'size_2': {
#        'distribution': 'q_log_uniform',
#        'q': 8,
#        'min': np.log(64),
#        'max': np.log(256)
#     },
#      'size_3': {
#        'distribution': 'q_log_uniform',
#        'q': 8,
#        'min': np.log(64),
#        'max': np.log(256)
#     },
#      'size_4': {
#        'distribution': 'q_log_uniform',
#        'q': 8,
#        'min': np.log(64),
#        'max': np.log(256)
#     },
#      'size_5': {
#        'distribution': 'q_log_uniform',
#        'q': 8,
#        'min': np.log(64),
#        'max': np.log(256)
#     },
#     'dropout': {
#       'distribution': 'uniform',
#        'min': 0,
#        'max': 0.6
#     },
#     'learning_rate':{
#          #uniform distribution between 0 and 1
#          'distribution': 'uniform', 
#          'min': 0,
#          'max': 0.1
#      }
# }

parameters_dict = {
    'epochs': {
       'distribution': 'int_uniform',
       'min': 20,
       'max': 100
    },
    # for build_dataset
     'batch_size': {
       'values': [32, 64, 96, 128]
    },
    'size_1': {
       'distribution': 'int_uniform',
       'min': 64,
       'max': 256
    },
    'size_2': {
       'distribution': 'int_uniform',
       'min': 64,
       'max': 256
    },'size_3': {
       'distribution': 'int_uniform',
       'min': 64,
       'max': 256
    },'size_4': {
       'distribution': 'int_uniform',
       'min': 64,
       'max': 256
    },'size_5': {
       'distribution': 'int_uniform',
       'min': 64,
       'max': 256
    },
    'dropout': {
      'distribution': 'uniform',
       'min': 0,
       'max': 0.6
    },
    'learning_rate':{
         #uniform distribution between 0 and 1
         'distribution': 'uniform', 
         'min': 0,
         'max': 0.1
     }
}

# append parameters to sweep config
sweep_config['parameters'] = parameters_dict 

# login to wandb----------------
wandb.init(project="Astro101_Project_v2", entity="oscarscholin")

# initialize sweep agent
sweep_id = wandb.sweep(sweep_config, project='Astro101_Project_v2', entity="oscarscholin")
wandb.agent(sweep_id, train, count=100)

#train_manual()

Num GPUs Available:  0


wandb: Currently logged in as: oscarscholin. Use `wandb login --relogin` to force relogin


wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. Additional properties are not allowed ('goal' was unexpected)
wandb: WARNING   Violation 2. 'val_accuracy' is not of type 'object'
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: etr6zhlb
Sweep URL: https://wandb.ai/oscarscholin/Astro101_Project_v2/sweeps/etr6zhlb


wandb: Waiting for W&B process to finish... (success).
wandb: Synced confused-terrain-84: https://wandb.ai/oscarscholin/Astro101_Project_v2/runs/2uwz5yx9
wandb: Synced 6 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20221126_233314-2uwz5yx9/logs
wandb: Agent Starting Run: 7hdh0gud with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0.5285490493519184
wandb: 	epochs: 59
wandb: 	learning_rate: 0.06915066450066502
wandb: 	size_1: 109
wandb: 	size_2: 165
wandb: 	size_3: 211
wandb: 	size_4: 166
wandb: 	size_5: 189


2022-11-26 23:33:31.924275: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


wandb: ERROR Run 7hdh0gud errored: ValueError('Failed to convert a NumPy array to a Tensor (Unsupported object type float).')
wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 47k4a9zs with config:
wandb: 	batch_size: 64
wandb: 	dropout: 0.09000704458630916
wandb: 	epochs: 66
wandb: 	learning_rate: 0.032069285626610024
wandb: 	size_1: 239
wandb: 	size_2: 240
wandb: 	size_3: 68
wandb: 	size_4: 159
wandb: 	size_5: 155
wandb: Ctrl + C detected. Stopping sweep.


Error in callback <function _WandbInit._pause_backend at 0x7fdeb161b430> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

In [4]:
train_x_ds = np.load(os.path.join(DATA_DIR, 'train_x_ds.npy'), allow_pickle=True)
print(train_x_ds)

Error in callback <function _WandbInit._resume_backend at 0x7fdedd5d35e0> (for pre_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

[[3.82409288414328e-08 0.328649973347305 0.479343119847827 ...
  7.32369860751236e-07 0.0037805501754816 'ROT']
 [0.0 0.0309871183707678 0.488597241876524 ... 0.0008473113844437
  0.0372233502693875 'SR']
 [0.999933027236402 0.574041866534529 0.574839699476286 ...
  2.53752680726072e-06 0.024554208231208 'EA']
 ...
 [6.710681392966011e-15 0.476331049882054 0.653910156500738 ...
  9.13362445145453e-07 0.240955312887744 'EB']
 [0.999972548386111 0.52285266759497 0.662324112878175 ...
  8.03309459320127e-07 0.042048206813697 'EW']
 [0.933968167464518 0.595885148359555 0.470090464092983 ...
  3.22696000356963e-07 0.0140796732878937 'EA']]
Error in callback <function _WandbInit._pause_backend at 0x7fdeb161b430> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe